<font color = "blue">

# Introduction 


Hello people! Welcome to my kernel. In this kernel I am going to do EDA and visualizations with using Countries of The World Data. I decided to use this dataset because in this kernel I am going to only use plotly as much as I can and I want a dataset that have a lot of numerical features.


## Schedule
1. Introduction
1. Importing the Data and Libraries
    * Importing Libraries
    * Imporing the Data
1. Diagnosing Data
1. Cleaning Data
    * comma2dot Function
    * type_converter Function
1. Correlation Examination and Visualization
    * Correlation Diagnosing
    * Correlation Visualization with using Heatmap
1. Simple Data Analyses
    * Simple Data Analyses with Population Data
    * Simple Data Anaylses with Coastline (coast/area) Data
    * Simple Data Anaylses with Net Migration Data
    * Simple Data Anaylses with Infant Mortality
    * Simple Data Anaylses with GDP per capita
1. Other Visualizations and Analyses
    * Median Visualizations using Box Plot


# Importing the Data And Libraries

In this section I am going to import the data and libraries that I need.

## Importing Libraries

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns #Seaborn visualization library
import matplotlib.pyplot as plt #Matplotlib visualization library



"""
Plotly Visualization Library / Main Tool of This Kernel
"""
import plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px




# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Importing the Data

In [1]:
data = pd.read_csv("/kaggle/input/countries-of-the-world/countries of the world.csv")

# Diagnosing Data

In this section I am going to take a look at the data because If I will do EDA, I need to know what type of data do I work with
I use *head* *tail* *columns* *dtypes* and *info* for diagnosing data. Time to start

In [1]:
data.head()

In [1]:
data.tail()

In [1]:
data.columns

In [1]:
data.dtypes

In [1]:
data.info()

# Cleaning Data

In this section I am going to clean the data because we have to clean data if we want to do simple analyses. I have to drop nan values because I can not work with them. But before dropping them, I want to visualize them, for this I am going to use missingno visualization library.


In [1]:
#Matrix plot
import missingno as missin #I am importing the library that I use
data = pd.read_csv("/kaggle/input/countries-of-the-world/countries of the world.csv")
fig,ax = plt.subplots(figsize=(8,8)) #I am creating my figure
missin.matrix(data,ax=ax,sparkline=False) # I am creating the plot
plt.show() 

In [1]:
#Missingno bar plot
fig,ax = plt.subplots(figsize=(8,8))
missin.bar(data,ax=ax)
plt.show()

Although nan values are rarely seen in the data set I have to drop nan values

In [1]:
data.dropna(inplace=True)

Now I realized some values have commas but in programming if a value is float you have to use dot. So I am going to write a function that converts commas into dots

In [1]:
def comma2dot(dataframe,feature):
    
    dirty_data = [i for i in dataframe[feature]]
    clean_data = []
    for i in dirty_data:
        
        if "," in i:
            i = i.replace(",",".")
        
        clean_data.append(i)
        
    dataframe[feature] = clean_data    
    return dataframe

In [1]:
comma_including_ftrs = ["Net migration","Infant mortality (per 1000 births)","Literacy (%)"
                          ,"Phones (per 1000)","Arable (%)","Crops (%)","Coastline (coast/area ratio)"
                          ,"Birthrate","Deathrate"]

for cm in comma_including_ftrs :
    
    data = comma2dot(data,cm)

We converted commas into dots and now I am going to write a function that helps me convert data types of features 

In [1]:
def type_converter(dataframe,feature,dtype="float"):
    
    if dtype == "float":
        dataframe[feature] = dataframe[feature].astype(float)
    
    elif dtype == "int":
        dataframe[feature] = dataframe[feature].astype(int)
    
    elif dtype == "string" or dtype == "str":
        dataframe[feature] = dataframe[feature].astype(str)
    
    return dataframe

Our function is ready, time to converting

In [1]:
data.columns
features = ["Population","Area (sq. mi.)","Net migration","Infant mortality (per 1000 births)","Literacy (%)",
           "Phones (per 1000)","Arable (%)","Crops (%)","Coastline (coast/area ratio)","Birthrate","Deathrate"]

for ftr in features:
        
    data = type_converter(data,ftr)
    

We have cleaned the data, now we are ready to do some analyses! 

# Correlation Examination and Visualization

In this section I am going to examine correlation between numerical features and visualize them with using different visualization tools. First we need to see correlation. For this I am going to use .corr() function. 

In [1]:
data.corr()

And now I am going to create a heatmap 

In [1]:
fig,ax = plt.subplots(figsize=(10,10))
sns.heatmap(data.corr(),linewidth=1,annot=True,linecolor="Black",fmt=".1f")
plt.title("Correlation Map",fontsize=15)
plt.show()

As we can see there are strong correlations betweens some features in our data but we examine them in the next section. 

And now I am going to create a network plot that shows us correlations.

In [1]:
#We are going to use this threshold for filtering correlation values
threshold = 0.6 

#Dataframe stacking
corr = data.corr()
stacked = corr.stack().reset_index()
stacked.columns = ["feature1","feature2","cor"]

#Filtering correlation values
stacked = stacked.loc[(stacked.feature1 != stacked.feature2) & (stacked.cor >= threshold)]
stacked = stacked.reset_index()
stacked


In [1]:
import networkx as nx
network_graph = nx.from_pandas_edgelist(stacked,"feature1","feature2")
nx.draw_circular(network_graph,with_labels=True,node_color="blue",node_size=20,edge_color="#C95DD9",linewidths=13)


We can see strong positive correlations using this networkx plot.


# Simple Data Analyses

In this section I am going to examine relationships between some features and frequency of features. I am going to use line plots,scatters,histograms 

## Simple Data Analyses with Population Data

In this sub-section I am going to examine relationships between population and some other features. If we take a look at our heatmap we can say there is only one strong relationship between Population and other features. It is Population-Area Relationship I am going to visualize it but before I am going to use histogram for frequency visualizations.


### Histogram Visualizations


In [1]:
#Histogram
trace1 = go.Histogram(x=data["Population"]
                     ,name="Population"
                     ,marker=(dict(color="rgba(234,12,43,0.8)"))
                     ,opacity=0.7)

layout = go.Layout(title="Population Histogram"
                  ,xaxis=dict(title="Population")
                  ,yaxis=dict(title="Counts"))

figure = go.Figure(data=trace1,layout=layout)

iplot(figure)

### Correlation Visualizations with using Line Plot and Scatter Plot

In this sub section I am going to visualize Population-Area with using Line Plot and Scatter Plot. First I am going to use Line plot.


In [1]:
normalized_df = data.copy()
normalized_df["Population"] = normalized_df["Population"]/max(normalized_df["Population"])
normalized_df["Area (sq. mi.)"] = normalized_df["Area (sq. mi.)"]/max(normalized_df["Area (sq. mi.)"])

new_indexes=normalized_df["Population"].sort_values(ascending=False).index.values
normalized_df = normalized_df.reindex(new_indexes)

trace1 = go.Scatter(x=normalized_df["Population"],
                    y=normalized_df["Area (sq. mi.)"],
                    mode="markers",
                    name = "Population",
                    text = data["Country"])


traces = [trace1]

layout = dict(title="Relationship Between Population and Area Data",xaxis=dict(title="Population",ticklen=5),yaxis=dict(title="Area",ticklen=5))

fig = dict(data=traces,layout=layout)

normalized_df.head()

In [1]:
iplot(fig)

In [1]:
trace1 = go.Scatter(x=normalized_df["Population"],
                    y=normalized_df["Area (sq. mi.)"],
                    mode="lines",
                    name = "Population",
                    text = data["Country"])


traces = [trace1]

layout = dict(title="Relationship Between Population and Area Data",xaxis=dict(title="Population ",ticklen=5))

fig = dict(data=traces,layout=layout)

iplot(fig)

### 10 Most Populous Countries Examination and Visualization

In this section I am going to determine 10 most populous countries and after that I am going to visualize them with using bar plot and line plot. 

In [1]:
#Determining 10 most crowded country
crowded_countries_index = data["Population"].sort_values(ascending=False).index.values
crowded_countries = data.reindex(crowded_countries_index)
crd = crowded_countries.head(10)
crd

We have determined the 10 most populous countries so we are ready to visualization.

In [1]:
trace1 = go.Bar(x = crd.Country,
                y=crd.Population,
               name="Country Name",
               text = crd.Country,
               marker = dict(color="rgba(54,170,120,0.8)",line = dict(color="rgb(0,0,0)",width=1.5)))

trc_all = [trace1]

layout = go.Layout(dict(title="10 Most Populous Country in the World",xaxis = dict(title="Countries"),
                       yaxis=dict(title="Population")))

figure = go.Figure(data=trc_all,layout=layout)

iplot(figure)

## Simple Data Analyses with Using Coastline(coast/area ratio)

In this sub-section, I am going to do some simple data analyses wiht using coastline data. If we go back to Correlation Section, we can see there is not a correlation between coastline data and other numerical features, so in this section, I am not going to do some correlation visualizations. First, I am going to determine Top 10 of Coastline data and then I am going to visualize them using Bar plot and Pie Chart.

### Determining and Visualizing Coastline Data

In [1]:
#Histogram
trace1 = go.Histogram(x=data["Coastline (coast/area ratio)"]
                     ,name="Coastline Ratio"
                     ,marker=(dict(color="rgba(234,12,212,0.8)"))
                     ,opacity=0.7)

layout = go.Layout(title="Coastline Histogram"
                  ,xaxis=dict(title="Coastline")
                  ,yaxis=dict(title="Counts"))

figure = go.Figure(data=trace1,layout=layout)

iplot(figure)

In [1]:
coast_line_index = data["Coastline (coast/area ratio)"].sort_values(ascending=False).index.values
coastline_data = data.reindex(coast_line_index)
cst = coastline_data.head(10)
cst

### Visualization using Bar Plot

In [1]:
#Bar plot

trace1 = go.Bar(x=cst.Country
      ,y=cst["Coastline (coast/area ratio)"]
      ,name="Coastline"
      ,text=cst.Region
      ,marker=dict(color="rgba(122,123,43,0.7)",line=dict(color="rgb(0,0,0)",width=1.5)))

traces = [trace1]

layout = go.Layout(dict(title="Top 10 Countries of Coastline"),xaxis=dict(title="Countries"),yaxis=dict(title="Coastline Ratio"))

fig = go.Figure(data=traces,layout=layout)

iplot(fig)

### Visualization using Pie Chart


In [1]:
trace = go.Pie(values=cst["Coastline (coast/area ratio)"]
              ,labels=cst.Country
              ,name="Countries"
              ,hoverinfo="label+percent+name")

layout = go.Layout(dict(title="10 Countries with the Most Coastline"))

figure = go.Figure(data=trace,layout=layout)

iplot(figure)

## Simple Data Anaylses with Net Migration Rate

In this section I am going to use net migration rate but before using this data I am going to explain what is net migration rate

The Net Migration Rate is divided into two. Positive migration rate and Negative migration rate. If the  migration rate is positive, it means  the number of people that come to the country is more than the number of  people that go from the country. But If the number of people that go from the country is more than the number of  people that come to the country, That country's migration rate will be negative.

We can go back our subject. If we take a look at the correlation table we can see there is not a strong correlation between migration data and other features so, I am not going to do correlation visualizations.

First I am going to do histogram visualization, Then, I am going to create new two data frame. Positive Net Migration Countries and Negative Net Migration Countries. 

In [1]:
#Histogram
trace1 = go.Histogram(x=data["Net migration"]
                     ,name="Net Migration"
                     ,marker=(dict(color="rgba(234,231,43,0.8)"))
                     ,opacity=0.7)

layout = go.Layout(title="Net Migration Histogram"
                  ,xaxis=dict(title="Net Migration")
                  ,yaxis=dict(title="Counts"))

figure = go.Figure(data=trace1,layout=layout)

iplot(figure)

In [1]:
#Creating dataframes
positive_mig = data[data["Net migration"]>0] 
negative_mig = data[data["Net migration"]<0]

#Sorting values
new_index = positive_mig["Net migration"].sort_values(ascending=False).index.values
positive_mig = positive_mig.reindex(new_index)

new_index = negative_mig["Net migration"].sort_values().index.values
negative_mig = negative_mig.reindex(new_index)

#Taking a look at negative data
positive_mig.head()

In [1]:
#taking a look at negative data
negative_mig.head()

We have created our dataframes and now we are ready to do visualizations. First I am going to visualize top 10 countries of dataframes using bar plot. And after that I am going to use pie chart. 

In [1]:
trace1 = go.Bar(x=positive_mig["Country"]
               ,y=positive_mig["Net migration"]
               ,name="Countries/Net Migration (Positive)"
               ,text=positive_mig.Region
               ,marker=dict(color="rgba(132,42,187,0.7)",line=dict(color="rgb(0,0,0)",width=1.5)))


trace2 = go.Bar(x=negative_mig["Country"]
               ,y=negative_mig["Net migration"]
               ,name="Countries/Net Migration (Negative)"
               ,text=negative_mig.Region
               ,marker=dict(color="rgba(42,230,187,0.7)",line=dict(color="rgb(0,0,0)",width=1.5)))

layout = go.Layout(dict(title="Top 10 Countries of Datasets"
                        ,xaxis=dict(title="Countries")
                        ,yaxis=dict(title="Net Migration")))


figure = go.Figure(data=[trace1,trace2],layout=layout)

iplot(figure)

## Simple Data Anaylses with Infant Mortality

In this section I am going to examine infant mortality data. If we check our correlation heatmap, we can see there are 3 strong correlations between Infant Mortality and other numerical features
    
1. Infant Mortality - GDP Per Capita (-0.6)
1. Infant Mortality - Literacy (-0.76)
1. Infant Mortality - Phones(per 1000) (-0,99)

At the first of this section I am going to do histogram visualization. Then, I am going to visualize these correlations with using scatter plot.

In [1]:
#Histogram
trace1 = go.Histogram(x=data["Infant mortality (per 1000 births)"]
                     ,name="Infant Mortality"
                     ,marker=(dict(color="rgba(0,238,90,1)"))
                     ,opacity=0.7)

layout = go.Layout(title="Infant Mortality Histogram"
                  ,xaxis=dict(title="Infant Mortality")
                  ,yaxis=dict(title="Counts"))

figure = go.Figure(data=trace1,layout=layout)

iplot(figure)

In [1]:
# Scatter Plot using Infant Mortality - GDP Per Capita

trace1 = go.Scatter(x=data["Infant mortality (per 1000 births)"]
                   ,y=data["GDP ($ per capita)"]
                   ,mode="markers"
                   ,name="Infant Mortality - GDP Per Capita"
                   ,text=data.Country)

layout = go.Layout(dict(title="Infant Mortality - GDP Per Capita"
                        ,xaxis=dict(title="Infant Mortality")
                        ,yaxis=dict(title="GDP Per Capita")))

figure = go.Figure(data=trace1,layout=layout)

iplot(figure)

In [1]:
#Infant Mortality - Literacy Scatter Plot
trace1 = go.Scatter(x=data["Infant mortality (per 1000 births)"]
                   ,y=data["Literacy (%)"]
                   ,mode="markers"
                   ,name="Infant Mortality - Literacy"
                   ,text=data.Country)

layout = go.Layout(dict(title="Infant Mortality - Literacy"
                        ,xaxis=dict(title="Infant Mortality")
                        ,yaxis=dict(title="Literacy")))

figure = go.Figure(data=trace1,layout=layout)

iplot(figure)

In [1]:
#Infant Mortality - Phones (per 1000)
trace1 = go.Scatter(x=data["Infant mortality (per 1000 births)"]
                   ,y=data["Phones (per 1000)"]
                   ,mode="markers"
                   ,name="Infant Mortality - Phones"
                   ,text=data.Country)

layout = go.Layout(dict(title="Infant Mortality - Phones"
                        ,xaxis=dict(title="Infant Mortality")
                        ,yaxis=dict(title="Phones")))

figure = go.Figure(data=trace1,layout=layout)

iplot(figure)

Correlation visualizations are over and now I am going to determine Top 10 Countries by Infant Mortality and Last 10 Countries. 

In [1]:
#Sorting values
sorted_indexes = data["Infant mortality (per 1000 births)"].sort_values(ascending=False).index.values
infant_df = data.reindex(sorted_indexes)

top5 = infant_df.tail()
last5 = infant_df.head()

#Taking a look at top5 data
top5

In [1]:
#taking a look at last5 data
last5

We are ready to do some visualizations.

In [1]:
trace1 = go.Bar(x=top5.Country
               ,y=top5["Infant mortality (per 1000 births)"]
               ,name="Top 5 Countries by Infant Mortality"
               ,text=top5.Region
               ,marker=dict(color="rgba(213,21,90,0.7)",line=dict(color="rgb(0,0,0)",width=1.5)))

trace2 = go.Bar(x=last5.Country
               ,y=last5["Infant mortality (per 1000 births)"]
               ,name="Last 5 Countries by Infant Mortality"
               ,text=last5.Region
               ,marker=dict(color="rgba(32,98,202,0.7)",line=dict(color="rgb(0,0,0)",width=1.5)))

traces = [trace1,trace2]

layout = go.Layout(title="Infant Mortality Bar Plot"
                  ,xaxis=dict(title="Country Names")
                  ,yaxis=dict(title="Infant Mortality"))

figure = go.Figure(data=traces,layout=layout)

iplot(figure)

And I want to make a pie chart. But before that I have to concat two dataframes.

In [1]:
#Concating dataframes
t5_nd_l5 = pd.concat([top5,last5],axis=0)
t5_nd_l5

In [1]:
trace1 = go.Pie(values=t5_nd_l5["Infant mortality (per 1000 births)"]
               ,labels=t5_nd_l5.Country
               ,name="Infant Mortality per 1000 births"
               ,hoverinfo= "label+percent+name")


layout = go.Layout(dict(title="Top 5 and Last 5 countries by Infant Mortality"))

figure = go.Figure(data=trace1,layout=layout)

iplot(figure)

## Simple Data Anaylses with GDP per capita

In this section I am going to do some simle data analyses using gdp per capita data. At the first of this section I am going to examine correlations between GDP per capita data and other numerical features.

There are four strong correlations between GDP per capita data and other numerical features.
1. GDP - Infant Mortality (-0.6)
1. GDP - Literacy (0.5)
1. GDP - Phones (per 1000) (0.88)
1. GDP - Birthrate (-0.65)
 
I am going to visualize them using scatter plot. But before, I am going to create a histogram plot


In [1]:
#Histogram
trace1 = go.Histogram(x=data["GDP ($ per capita)"]
                     ,name="GDP per capita"
                     ,marker=(dict(color="rgba(181,118,54,1)"))
                     ,opacity=0.9)

layout = go.Layout(title="GDP per capita Histogram"
                  ,xaxis=dict(title="GDP Per capita")
                  ,yaxis=dict(title="Counts"))

figure = go.Figure(data=trace1,layout=layout)

iplot(figure)

In [1]:
#GDP - Infant Mortality (-0.6) Visualization

trace1 = go.Scatter(x=data["Infant mortality (per 1000 births)"]
                   ,y=data["GDP ($ per capita)"]
                   ,name="Infant Mortality -  GDP Per Capita ",
                    mode = "markers"
                   ,text=data.Country
                   ,marker =dict(color="rgba(68,234,187,0.7)",line=dict(color="rgb(0,0,0)",width=1.5)))

layout = go.Layout(dict(title="Infant Mortality - GDP Per Capita Visualization",
                        xaxis=dict(title="Infant Mortality"),
                        yaxis=dict(title="GDP Per Capita")))

figure = go.Figure(data=trace1,layout=layout)

iplot(figure)

In [1]:
#GDP - Literacy (0.5) Visualization

trace1 = go.Scatter(x=data["Literacy (%)"]
                   ,y=data["GDP ($ per capita)"]
                   ,name="Literacy -  GDP Per Capita ",
                    mode = "markers"
                   ,text=data.Country
                   ,marker =dict(color="rgba(68,234,187,0.7)",line=dict(color="rgb(0,0,0)",width=1.5)))

layout = go.Layout(dict(title="Literacy - GDP Per Capita Visualization",
                        xaxis=dict(title="Literacy"),
                        yaxis=dict(title="GDP Per Capita")))

figure = go.Figure(data=trace1,layout=layout)

iplot(figure)

In [1]:
#GDP - Phones (0.88) Visualization

trace1 = go.Scatter(x=data["Phones (per 1000)"]
                   ,y=data["GDP ($ per capita)"]
                   ,name="Phones -  GDP Per Capita ",
                    mode = "markers"
                   ,text=data.Country
                   ,marker =dict(color="rgba(68,234,187,0.7)",line=dict(color="rgb(0,0,0)",width=1.5)))

layout = go.Layout(dict(title="Phones (per 1000) - GDP Per Capita Visualization",
                        xaxis=dict(title="Phones"),
                        yaxis=dict(title="GDP Per Capita")))

figure = go.Figure(data=trace1,layout=layout)

iplot(figure)

In [1]:
#GDP -Birthrate (-0.65) Visualization

trace1 = go.Scatter(x=data["Birthrate"]
                   ,y=data["GDP ($ per capita)"]
                   ,name="Birthrate -  GDP Per Capita ",
                    mode = "markers"
                   ,text=data.Country
                   ,marker =dict(color="rgba(68,234,187,0.7)",line=dict(color="rgb(0,0,0)",width=1.5)))

layout = go.Layout(dict(title="Birthrate - GDP Per Capita Visualization",
                        xaxis=dict(title="Birthrate"),
                        yaxis=dict(title="GDP Per Capita")))

figure = go.Figure(data=trace1,layout=layout)

iplot(figure)

We have visualized our correlation knowledges. And now I am going to determine Top 10 Countries by GDP Per Capita and Last 10 Countries by GDP Per capita.

In [1]:
#Determining 
top10_index = data["GDP ($ per capita)"].sort_values(ascending=False).index.values
last10_index = data["GDP ($ per capita)"].sort_values().index.values
top10_gdp = data.reindex(top10_index)
last10_gdp = data.reindex(last10_index)
top10_gdp= top10_gdp.iloc[:10]
last10_gdp = last10_gdp.iloc[:10]
top10_gdp

In [1]:
last10_gdp

In [1]:
#Visualizing
trace1 = go.Bar(x=top10_gdp["Country"]
               ,y=top10_gdp["GDP ($ per capita)"]
               ,name="Top 10 Countries by GDP per capita"
               ,text=top10_gdp.Region
               ,marker=(dict(color="rgba(213,52,5,0.8)",line=dict(color="rgb(0,0,0)",width=1.5))))


trace2 = go.Bar(x=last10_gdp["Country"]
               ,y=last10_gdp["GDP ($ per capita)"]
               ,name="Last 10 Countries by GDP per capita"
               ,text=last10_gdp.Region
               ,marker=(dict(color="rgba(94,21,231,0.8)",line=dict(color="rgb(0,0,0)",width=1.5))))

layout = go.Layout(dict(title="Top and Last 10 Countries by GDP per capita")
                  ,xaxis=dict(title="Country"),yaxis=dict(title="GDP per capita"))

figure = go.Figure(data=[trace1,trace2],layout=layout)

iplot(figure)

# Other Visualizations and Analyses
In this chapter I am going to do different visualizations and analyses. I created this section because I want my kernel tidy. 

## Median Visualization using Box Plot
In this sub-section I am going to visualize median values using box plot. At the first, I am going to write a function that creates box plots.

In [1]:
#function creating
def boxplotCreator(dataFrame,yValueList,colorList,nameList):
    traces = []
    for yValue,color,name in (zip(yValueList,colorList,nameList)):
        trace1 = go.Box(y=dataFrame[yValue]
                       ,marker=(dict(color=color))
                       ,name=name)
        
        traces.append(trace1)
        
    
    figure = go.Figure(data=traces)
    iplot(figure)

Our function is ready. Now we can easily create a box plot.

In [1]:
numerical_features = [i for i in data.corr().columns]
numftr1 = numerical_features[:3]
numftr2 = numerical_features[3:6]
numftr3 = numerical_features[6:9]
numftr4 = numerical_features[9:]
colors = ["rgba(89,232,84,0.9)"
         ,"rgba(147,242,88,0.8)"
         ,"rgba(67,237,55,0.9)"
         ,"rgba(141,40,252,0.8)"
         ,"rgba(40,166,252,0.8)"
         ,"rgba(111,253,182,0.7)"
         ,"rgba(27,143,212,0.6)"
         ,"rgba(123,198,219,0.8)"
         ,"rgba(129,21,124,0.7)"
         ,"rgba(142,12,43,0.8)"
         ,"rgba(5,145,239,0.8)"
         ,"rgba(43,35,31,0.8)"]
colors1 = colors[:3]
colors2 = colors[3:6]
colors3 = colors[6:9]
colors4 = colors[9:]

boxplotCreator(data,numftr1,colors1,numftr1)

In [1]:
boxplotCreator(data,numftr2,colors2,numftr2)

In [1]:
boxplotCreator(data,numftr3,colors3,numftr3)

In [1]:
boxplotCreator(data,numftr4,colors4,numftr4)

## Geographic Visualization using Scatter_Geo

In this section I am going to do visualization using scatter geo however for using scatter_geo, we need coordinates of countries so, but I can not get coordinates all countries because it takes hours so, I decided to use these 10 countries.

United States of America  38 00 N, 97 00 W

United Kingdom 54 N 2 W 

Germany 51 N 9 E 

Turkey 39N 35E

Greece 39 N 22E 

Norway 62N 10E

South Africa 29S 24E

Netherlands 52.30 N 5.45 E 

Japan 36N 138 E

Canada 60N 95W

First I am going to create a new data frame


In [1]:
country_name = ["United States of America","United Kingdom","Germany",
               "Greece","Turkey","Norway","South Africa","Netherlands",
               "Japan","Canada"]

country_lat = [38,54,51,39,
              39,62,-29,52.30,36,60]

country_lon = [-97,-2,9,22,35,10,24,5.45,138,-95]

country_locations = pd.DataFrame(dict(Country=country_name,CountryLat=country_lat,CountryLon=country_lon))

country_locations

Our dataframe is ready so we are ready to visualize them.

In [1]:
#trace
trace1 = go.Scattergeo(lon=country_locations.CountryLon
                      ,lat=country_locations.CountryLat
                      ,text=country_locations.Country
                      ,name="text"
                      ,mode="markers")


layout = go.Layout(dict(title="Some Countries of the World"
                       ,hovermode="closest"
                       ,geo = dict(showframe=False,showland=True,showcoastlines=True,showcountries=True)))


iplot(go.Figure(data=trace1,layout=layout))

## Comparing Top 20 Countries by GDP using Parallel Coordinate Plot

* In this section, I am going to use pandas libraries' parallel coordinate plot. I would like to use all the countries but I do not want to be plot confusing so I have decided to use top 20 countries by GDP.

In [1]:
#I am preparing my data for use
GDP_index = data["GDP ($ per capita)"].sort_values(ascending=False).index.values
sorted_data = data.reindex(GDP_index)
sorted_data = sorted_data.head(20)
sorted_data.head()

Our data is ready, let's visualize it

In [1]:
cols = [i for i in data.corr().columns
        if not i== "Population" 
        and not i=="Area (sq. mi.)" and not i=="GDP ($ per capita)"
        and not i=="Phones (per 1000)"] # columns

fig,ax = plt.subplots(figsize=(12,10))
pd.plotting.parallel_coordinates(sorted_data, class_column='Country', cols=cols,colormap=plt.get_cmap("plasma"),linewidth=3)
plt.xlabel("Features")
plt.title("Parallel Cordinates Plot",fontsize=15)
plt.xticks(rotation=90)
plt.show()

## Positive Correlation Visualization using 3D Scatter

Before the finishing this kernel, I want to add a sub-section about comparing three or four features
using 3D Scatter plot. In this section I am going to comparing three features using 3D scatter.

First I am going to take a look at the correlation data for remembering

In [1]:
data.corr()

I decided to use these three features: Phones, GDP per capita and Literacy.

In [1]:
#3D Scatter

trace = go.Scatter3d(x=data["Phones (per 1000)"]
                    ,y=data["GDP ($ per capita)"]
                    ,z=data["Literacy (%)"]
                    ,mode="markers"
                    ,marker=dict(color="rgba(42,153,34,0.7)")
                    ,name="Phones | GDP | Literacy"
                    ,text=data.Country)

layout = go.Layout(dict(title="Comparing Three Feauteres - Phones | GDP | Literacy"))
figure = go.Figure(data=trace,layout=layout)

iplot(figure)

# The End

Thanks for your attention. If I have mistakes and language problems please contact with me or write a comment for this kernel.

PS : This kernel have not completed yet, I am going to update this kernel. 

Have a good day!